In [1]:
# %cd content
!ls
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

sample_data
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%ls
%cd drive
%cd My Drive
%cd Crowding_project

drive/  sample_data/
/content/drive
/content/drive/My Drive
/content/drive/My Drive/Crowding_project


# Import libraries

In [3]:
import tensorflow as tf
import numpy as np
import os

import keras
from keras.models import Sequential
from keras.layers import Input,Flatten, Conv2D, MaxPooling2D,UpSampling2D,add, concatenate
from keras.layers.convolutional import Convolution2D
from keras.layers.core import Dropout, Activation
from keras.layers.pooling import GlobalAveragePooling2D
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras import optimizers


# For imshow the images
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt

Using TensorFlow backend.


# Preparing the dataset 

In [4]:
#######################################################
# Import training input
train_images = np.load('Npy_data/Train_image.npy')
train_labels = np.load('Npy_data/Train_den.npy')
print('training preparation is completed')

#######################################################
# Import validation input
val_images = np.load('Npy_data/Val_image.npy')
val_labels = np.load('Npy_data/Val_den.npy')
print('Val preparation is completed')

# Normalized data
TLs_max = train_labels.max()
TLs_min = train_labels.min()

train_images = (train_images - train_images.min()) / (train_images.max() - train_images.min())
train_labels = (train_labels - train_labels.min()) / (train_labels.max() - train_labels.min())


training preparation is completed
Val preparation is completed


# Hyperparameter

In [0]:
factor =3         # for control number of filter in the network
num_epoch = 50
batch_size = 1

# Slave network architecture

In [6]:
x_p = tf.placeholder(shape=[None,480,640,3],dtype=tf.float32)
y_p = tf.placeholder(shape=[None,480,640,1],dtype=tf.float32)
conv_11_p = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(x_p)
conv_12_p = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_11_p)
conv_13_p = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_12_p)
pool_1_p  = MaxPooling2D(pool_size=(2,2))(conv_13_p)

conv_21_p = Conv2D(16*factor, (3, 3), activation='relu', padding='same',trainable = True)(pool_1_p)
conv_22_p = Conv2D(16*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_21_p)
pool_2_p  = MaxPooling2D(pool_size=(2,2))(conv_22_p)

conv_31_p = Conv2D(32*factor, (3, 3), activation='relu', padding='same',trainable = True)(pool_2_p)
conv_32_p = Conv2D(32*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_31_p)
pool_3_p  = MaxPooling2D(pool_size=(2,2))(conv_32_p)

conv_41_p = Conv2D(64*factor, (3, 3), activation='relu', padding='same',trainable = True)(pool_3_p)
conv_42_p = Conv2D(64*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_41_p)



up_5_p   = UpSampling2D(size = (2,2))(conv_42_p)
# conv_51_p = Conv2D(32*factor, (3, 3), activation='relu', padding='same',trainable = True)(concatenate([up_5_p, conv_32_p], axis=-1))
conv_51_p = Conv2D(32*factor, (3, 3), activation='relu', padding='same',trainable = True)(up_5_p)
conv_52_p = Conv2D(32*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_51_p)
out_3                  = conv_52_p


up_6_p   = UpSampling2D(size = (2,2))(conv_52_p)
# conv_61_p = Conv2D(16*factor, (3, 3), activation='relu', padding='same',trainable = True)(concatenate([up_6_p, conv_22_p], axis=-1))
conv_61_p = Conv2D(16*factor, (3, 3), activation='relu', padding='same',trainable = True)(up_6_p)
conv_62_p = Conv2D(16*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_61_p)
out_2                  = conv_62_p


up_7_p  = UpSampling2D(size = (2,2))(conv_62_p)
# conv_71_p = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(concatenate([up_7_p, conv_13_p], axis=-1))
conv_71_p = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(up_7_p)
conv_72_p = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_71_p)
conv_73_p = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_72_p)
out_1                  = conv_73_p 

layer_final_p = Conv2D(1, (1, 1), activation='linear')(conv_73_p)

cost_p = tf.losses.mean_squared_error(y_p,layer_final_p)
auto_train_p = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost_p)


# For save the model weight in each layer in the slave network
saver_slave=tf.train.Saver()
save_dir_slave = 'Save_weighted/Slave_network/'
if not os.path.exists(save_dir_slave):
    os.makedirs(save_dir_slave)
save_path_slave = os.path.join(save_dir_slave, 'best_validation')





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# Restore the model weights

In [8]:
sess1 = tf.Session()
sess1.run(tf.global_variables_initializer())

# Restore the weight
saver_slave.restore(sess1, "Save_weighted/Slave_network/best_validation")
print("Restore model successfully")

for current_batch_index in range(0,len(val_images),batch_size):
  current_batch = val_images[current_batch_index:current_batch_index+batch_size,:,:,:]
  current_label = val_labels[current_batch_index:current_batch_index+batch_size,:,:,:]
  sess1_results = sess1.run([cost_p,out_1,out_2,out_3,layer_final_p],
                                      feed_dict = {x_p:current_batch,
                                                        y_p:current_label,
                                                          })
  P_1 = sess1_results[1]
  P_2 = sess1_results[2]
  P_3 = sess1_results[3]


INFO:tensorflow:Restoring parameters from Save_weighted/Slave_network/best_validation
Restore model successfully


# Master network architecture

In [0]:
o1 = tf.placeholder(shape=[None,480,640,8*factor],dtype=tf.float32)
o2 = tf.placeholder(shape=[None,480//2,640//2,16*factor],dtype=tf.float32)
o3 = tf.placeholder(shape=[None,480//4,640//4,32*factor],dtype=tf.float32)

x = tf.placeholder(shape=[None,480,640,3],dtype=tf.float32)
y = tf.placeholder(shape=[None,480,640,1],dtype=tf.float32)
conv_11 = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(concatenate([o1,x],axis =-1))
conv_12 = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_11)
conv_13 = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_12)
# pool_1  = MaxPooling2D(pool_size=(2,2))(concatenate([o1,conv_13],axis =-1))
pool_1  = MaxPooling2D(pool_size=(2,2))(conv_13)

conv_21 = Conv2D(16*factor, (3, 3), activation='relu', padding='same',trainable = True)(concatenate([o2,pool_1],axis =-1))
conv_22 = Conv2D(16*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_21)
# pool_2  = MaxPooling2D(pool_size=(2,2))(concatenate([o2,conv_22],axis =-1))
pool_2  = MaxPooling2D(pool_size=(2,2))(conv_22)

conv_31 = Conv2D(32*factor, (3, 3), activation='relu', padding='same',trainable = True)(concatenate([o3,pool_2],axis =-1))
conv_32 = Conv2D(32*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_31)
# pool_3  = MaxPooling2D(pool_size=(2,2))(concatenate([o3,conv_32],axis =-1))
pool_3  = MaxPooling2D(pool_size=(2,2))(conv_32)

conv_41 = Conv2D(64*factor, (3, 3), activation='relu', padding='same',trainable = True)(pool_3)
conv_42 = Conv2D(64*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_41)



up_5   = UpSampling2D(size = (2,2))(conv_42)
conv_51 = Conv2D(32*factor, (3, 3), activation='relu', padding='same',trainable = True)(up_5)
# conv_51 = Conv2D(32*factor, (3, 3), activation='relu', padding='same',trainable = True)(concatenate([up_5, conv_32], axis=-1))
conv_52 = Conv2D(32*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_51)
# out_3                  = conv_52


up_6   = UpSampling2D(size = (2,2))(conv_52)
conv_61 = Conv2D(16*factor, (3, 3), activation='relu', padding='same',trainable = True)(up_6)
# conv_61 = Conv2D(16*factor, (3, 3), activation='relu', padding='same',trainable = True)(concatenate([up_6, conv_22], axis=-1))
conv_62 = Conv2D(16*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_61)
# out_2                  = conv_62


up_7   = UpSampling2D(size = (2,2))(conv_62)
conv_71 = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(up_7)
# conv_71 = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(concatenate([up_7, conv_13], axis=-1))
conv_72 = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_71)
conv_73 = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_72)
# out_1                  = conv_73 

layer_final = Conv2D(1, (1, 1), activation='linear')(conv_73)

cost = tf.losses.mean_squared_error(y,layer_final)
auto_train = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)


# For save the model weight in each layer in the master network
saver_master=tf.train.Saver()
save_dir_master = 'Save_weighted/Master_network/'
if not os.path.exists(save_dir_master):
    os.makedirs(save_dir_master)
save_path_master = os.path.join(save_dir_master, 'best_validation')

# Restore the model weights

In [14]:
sess2 = tf.Session()
sess2.run(tf.global_variables_initializer())

# Restore the weight
saver_master.restore(sess2, "Save_weighted/Master_network/best_validation")
print("Restore model successfully")

Re_matrix = np.zeros([len(val_images),2])
for current_batch_index in range(0,10,batch_size):
  current_batch = val_images[current_batch_index:current_batch_index+batch_size,:,:,:]
  current_label = val_labels[current_batch_index:current_batch_index+batch_size,:,:,:]
  sess2_results = sess2.run([cost,layer_final],
            feed_dict = {x:current_batch,
                        y:current_label,
                        o1:P_1,
                        o2:P_2,
                        o3:P_3
                        })
  
  estimated_den_map = sess2_results[1]
  print(current_batch_index)
  
  ### Saving in CSV file
  Re_num = np.sum(np.squeeze(estimated_den_map[0,:,:,0]))
  GT_num = np.sum(np.squeeze(current_label[0,:,:,0]))
  Re_matrix[current_batch_index,1] = Re_num
  Re_matrix[current_batch_index,0] = GT_num
  

  ### For showing output
  show_image    = val_images[current_batch_index,:,:,:]
  show_den      = val_labels[current_batch_index,:,:,:]
  
  fig, (ax1, ax2, ax3) = plt.subplots(1,3)
  ax1.imshow(np.squeeze(show_image))
  ax1.set_title('Input image')

  ax2.imshow(np.squeeze(show_den))
  ax2.set_title('Ground truth')

  ax3.imshow(np.squeeze(estimated_den_map[0,:,:,0]))
  ax3.set_title('estimated density map')

  # show_index.zfill(4)
  plt.savefig('Output_image/Output_'+str(current_batch_index).zfill(4)+'.png')

np.savetxt("Experimental_result/back_ward.csv", Re_matrix, delimiter=",")
        

INFO:tensorflow:Restoring parameters from Save_weighted/Master_network/best_validation
Restore model successfully
0


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


1


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


2


...

6


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


7
8


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


9


In [0]:
sess2_results[1]